In [1]:
%run base.ipynb

# EPIC 1 - User Story DF-24
## Erstellung eines Notebooks und aufrufen der Daten per Dataland-API

Als QA-Lab-Team möchten wir ein Notebook erstellen, mit dem wir eine Anfrage an die Dataland-API senden, sodass wir anhand der Berichts-ID den Unternehmensbericht erhalten.

Beschreibung:<br>
Nutzen von Dataland Endpoint /data/nuclear-and-gas/companies/{companyId} im Nuclear & Gas Controller. Die Funktionalität wird außerhalb des Notebooks implementiert.

Akzeptanzkriterien:<br>
    - Notebook ist auf jedem Laptop ausführbar<br>
    - Anfrage an Dataland gibt Daten zurück

## 1.Schritt: Dataset mit Hilfe der Company_ID über die Dataland_API aufrufen
Mithilfe der Company_ID wird die Dataland_API aufgerufen. Nachdem Eintragen aus welcher Periode das Dataset sein soll, ist es möglich die Data_ID zu erlangen.

In [2]:
import dataland_qa_lab.dataland.get_data as qa
from dataland_qa_lab.utils import config

conf = config.get_config()
dataland_client = conf.dataland_client

data_id = qa.get_data_id_by_year(company_id="d6c56151-7fd0-481a-af9f-9fc45a16288b", year="2024")

dataset = qa.get_dataset_by_id(data_id=data_id)

print(dataset)

company_id='d6c56151-7fd0-481a-af9f-9fc45a16288b' reporting_period='2024' data=NuclearAndGasData(general=NuclearAndGasGeneral(general=NuclearAndGasGeneralGeneral(referenced_reports={'NFE-EWE-2023_english': CompanyReport(file_reference='86edb3cb151bfddf9ea694bc45203bd072fc9446c3d1fd42c7b14d51786ed7c8', file_name=None, publication_date=datetime.date(2023, 12, 7))}, nuclear_energy_related_activities_section426=ExtendedDataPointYesNo(value=<YesNo.NO: 'No'>, quality=<QualityOptions.ESTIMATED: 'Estimated'>, comment='', data_source=ExtendedDocumentReference(page='60', tag_name=None, file_name='NFE-EWE-2023_english', file_reference='86edb3cb151bfddf9ea694bc45203bd072fc9446c3d1fd42c7b14d51786ed7c8')), nuclear_energy_related_activities_section427=None, nuclear_energy_related_activities_section428=None, fossil_gas_related_activities_section429=None, fossil_gas_related_activities_section430=None, fossil_gas_related_activities_section431=None), taxonomy_aligned_denominator=NuclearAndGasGeneralTaxon

## 2.Schritt: Datensatz in PDF Reader umwandeln

In [3]:
import io

import pypdf

dataset_section426 = dataset.data.general.general.nuclear_energy_related_activities_section426
page_number = int(dataset_section426.data_source.page)

file_id = dataset_section426.data_source.file_reference
file_name = dataset_section426.data_source.file_name
tag_name = dataset_section426.data_source.tag_name
page = dataset_section426.data_source.page

pdf = dataland_client.documents_api.get_document(file_id)
pdf_stream = io.BytesIO(pdf)
pdf_reader = pypdf.PdfReader(pdf_stream)

## 3.Schritt: Daten aus dem Dokument extrahieren und in Text umwandeln

In [4]:
import dataland_qa_lab.dataland.data_extraction as qa_lab

relevant_page = qa_lab.get_relevant_page_of_pdf(page_number, pdf_reader)

text_of_page = qa_lab.extraxt_text_of_pdf(relevant_page)

section_426 = qa_lab.extract_section_426(text_of_page)

# section_426 = "No"
print(section_426)

No. The relevant documents indicate that the undertaking does not carry out, fund, or have exposures to research, development, demonstration, and deployment of innovative electricity generation facilities that produce energy from nuclear processes with minimal waste from the fuel cycle.
